### LCEL 인터페이스
사용자 정의 체인을 가능한 쉽게 만들 수 있도록, `Runnable`프로토콜을 구현했습니다. </br>

`Runnable`프로토콜은 대부분의 컴포넌트에 구현되어 있습니다. </br>

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다. 표준 인터페이스에는 다음이 포함됩니다.
- `stream` : 응답의 청크를 스트리밍합니다.
- `invoke` : 입력에 대해 체인을 호출합니다.
- `batch` : 입력 목록에 대해 체인을 호출합니다.
</br>

비동기 메소드도 있습니다.
- `astream` : 비동기적으로 응답의 청크를 스트리밍합니다.
- `ainvoke` : 비동기적으로 입력에 대해 체인을 호출합니다.
- `abatch` : 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- `astream_log` : 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다. 

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

# LangSmith 추적을 원하지 않을 경우
# logging.langsmith("CH01-Basic", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain을 생성합니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()

# 주어진 토픽에 대한 요약을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic}에 대하여 3문장으로 설명해줘.")

output_parser = StrOutputParser()

# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | output_parser

### stream : 실시간 출력
이 함수는 `chain.stream`메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""`인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`인자는 출력 버퍼를 즉시 비우도록 합니다.

In [5]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
answer = chain.stream({"topic" : "멀티모달"})

In [6]:
answer

<generator object RunnableSequence.stream at 0x169d29f30>

In [7]:
for token in answer:
    # 스트림에서 받은 데이터 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 형태의 다양한 미디어를 동시에 활용하여 정보를 전달하는 방식이다. 이는 텍스트, 이미지, 음성, 비디오 등을 모두 활용하여 사용자에게 보다 풍부한 경험을 제공해준다. 멀티모달을 통해 사용자는 자신에게 가장 이해하기 쉬운 형태의 정보를 선택하거나 조합하여 사용할 수 있다.

### invoke : 호출
`chain`객체의 `invoke`메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [ ]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
print(chain.invoke({"topic" : "ChatGPT"}))

ChatGPT는 고급 자연어 처리 모델로, 대화형 AI 애플리케이션에서 사용됩니다. 이 모델은 자연스러운 대화를 제공하며, 다양한 주제에 대해 질문에 답변을 하거나 의견을 공유할 수 있습니다. ChatGPT는 인간과 자연스럽게 상호작용하며, 사용자들이 딥 러닝 기술을 활용한 챗봇을 통해 더 효율적으로 정보를 얻을 수 있도록 도와줍니다.


### batch : 배치(단위 실행)
함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 인자로 받아, 각 딕셔너리에 있는 `topic`키의 값을 사용하여 일괄 처리를 수행합니다.

In [12]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
answer = chain.batch([{"topic" : "ChatGPT"}, {"topic" : "Instagram"}])

In [13]:
answer[0]

'ChatGPT는 인공지능 챗봇으로 자연어 처리를 통해 사용자와 대화하며 다양한 주제에 대해 정보를 제공합니다. 강력한 언어모델을 기반으로 자연스럽고 유창한 대화를 제공하여 사용자의 질문에 신속하고 정확한 답변을 제공합니다. 현대적인 기술을 활용하여 사용자와 상호작용하며 다양한 서비스에 활용됩니다.'

In [14]:
answer[1]

'Instagram은 사진 및 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 다양한 필터와 기능을 이용해 사진을 편집하고 친구들과 소통할 수 있다. 또한 해시태그를 활용하여 관심사나 주제별로 게시물을 찾을 수 있다.'

`max_concurrency`매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다. </br>

`config`딕셔너리는 `max_concurrency`키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되었습니다.

In [11]:
chain.batch(
    [
        {"topic" : "ChatGPT"},
        {"topic" : "Instagram"},
        {"topic" : "멀티모달"},
        {"topic" : "프로그래밍"},
        {"topic" : "머신러닝"},
    ],
    config={"max_concurrency" : 3},
)

['ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 사용자와 자연스럽게 대화하며 다양한 주제에 대해 대화할 수 있습니다. 이 모델은 기계 학습과 자연어 처리 기술을 이용하여 사람과 유사한 대화 경험을 제공합니다. ChatGPT를 통해 사용자는 질문에 답변을 받거나 자신의 이야기를 전달할 수 있어 소셜 스터디, 훈련, 상담 등에 다양하게 활용할 수 있습니다.',
 'Instagram은 사진 및 동영상 공유 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 사진과 동영상으로 공유하고 소통할 수 있는 서비스이다. 또한 해시태그를 통해 특정 주제나 관심사에 대한 컨텐츠를 찾을 수 있고, 팔로워를 통해 소셜 네트워크를 형성할 수 있다. 또한 인스타그램 스토리와 라이브 기능을 통해 실시간 소통과 콘텐츠 공유가 가능하다.',
 '멀티모달은 여러 가지 다른 형태의 입력이나 출력 수단을 결합하여 의사 소통이 가능한 시스템이다. 이는 음성, 텍스트, 이미지, 동영상 등 다양한 형태로 사용자와 상호작용할 수 있게 한다. 멀티모달 기술은 사용자 경험을 향상시키고 적응력을 높이는데 도움을 준다.',
 '프로그래밍은 컴퓨터에게 명령을 내리기 위해 사용되는 일련의 코드 작성 과정이다. 이는 알고리즘과 구문을 사용하여 원하는 작업을 수행하도록 컴퓨터에게 지시하는 것을 의미한다. 프로그래밍 언어에는 Python, Java, C++ 등 다양한 종류가 있으며, 이를 활용하여 소프트웨어 및 애플리케이션을 개발할 수 있다.',
 '머신러닝은 컴퓨터가 데이터를 학습하고 패턴을 인식하여 문제 해결이나 예측을 수행하는 인공지능 기술이다. 머신러닝은 주어진 데이터를 이용해 모델을 학습시키고, 새로운 데이터에 대한 예측이나 판단을 내릴 수 있다. 이를 통해 머신러닝은 자율주행 자동차, 음성 인식, 이미지 분류 등 다양한 분야에 활용되고 있다.']

In [ ]:
'''
# 비동기 메서드란?
비동기 메서드는 프로그램이 특정 작업을 수행하고 있을 때, 그 작업이 완료될 때까지 프로그램 전체를 멈추지 않고 다른 작업을 계속 수행할 수 있게 해주는 방법입니다.
즉, 작업의 완료를 기다리는 동안 다른 코드의 실행이 가능합니다.

# 일상의 예 : 커피숍
비동기 메서드를 커피숍에서의 상황으로 비유해보겠습니다. 여러분이 카페에 가서 커피를 주문했다고 상상해 봅시다.
- 동기 방식(Synchronous) : 바리스타에게 커피를 주문한 후, 커피가 나올 때까지 그 자리에서 기다립니다. 이 동안 다른 일을 하지 못하고 오로지 커피만을 기다리는 상태입니다.
- 비동기 방식(Asynchronous) : 바리스타에게 커피를 주문한 후, 번호표를 받고 자리에 앉아 다른 일(예: 책 읽기, 친구와 대화하기)을 합니다. 커피가 준비되면 번호가 호출되고, 이때 커피를 받으러 갑니다.

# 프로그래밍에서의 예
웹 서버를 개발할 때 비동기 메서드는 매우 유용하게 사용됩니다.
예를 들어, 사용자가 데이터베이스에서 정보를 요청하는 상황을 생각해 볼 수 있습니다.
- 동기 방식 : 서버는 데이터베이스의 응답을 기다리면서 다른 요청을 처리하지 못하고 멈춰 있습니다. 이는 효율성이 떨어질 수 있습니다.
- 비동기 방식 : 서버는 데이터를 요청하고, 그 응답을 기다리는 동안 다른 사용자의 요청을 계속해서 처리할 수 있습니다. 데이터베이스에서 응답이 오면, 그때 해당 작업을 완료합니다.

# 장점
1. 효율성 :
비동기 메서드는 시스템 자원을 보다 효율적으로 사용할 수 있게 합니다. 응답을 기다리는 동안에도 다른 작업을 처리할 수 있기 때문에, 전체적인 처리량과 응답 시간을 개선할 수 있습니다.
2. 사용자 경험 :
사용자는 작업이 백그라운드에서 처리되는 동안 다른 활동을 계속할 수 있어, 애플리케이션 반응성이 좋아집니다.

# 결론
비동기 메서드는 특히 웹 개발, 앱 개발 등에서 중요한 역할을 합니다. 다양한 작업을 동시에 처리할 수 있는 능력 덕분에, 프로그램이 더 빠르고 효율적으로 동작할 수 있게 도와줍니다.
'''

### async stream : 비동기 스트림
함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다. </br>

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print`함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.

In [15]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic" : "YouYue"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouYue란 중국의 글로벌 온라인 도서관 플랫폼으로, 수백만 권의 전자책과 웹툰을 제공합니다. 다양한 장르와 언어로 된 콘텐츠를 제공하여 독자들에게 다채로운 즐거움을 주고 있습니다. 또한 사용자들은 회원가입 후 손쉽게 전자책을 대여하고 읽을 수 있습니다.

### async invoke : 비동기 호출
`chain`객체의 `ainvoke`메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커)라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.

In [ ]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic" : "NVDA"})

In [ ]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

### async batch : 비동기 배치
함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다. </br>

이 예시에서는 `chain`객체의 `abatch`메서드를 사용하여 `topic`에 대한 작업을 비동기적으로 처리하고 있습니다. </br>

`await`키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [ ]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic" : "YouTue"}, {"topic" : "Instagram"}, {"topic" : "Facebook"}]
)

In [ ]:
# 비동기로 처리되는 일괄 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process